# Testing code

# Importing stuff

In [69]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import warnings

warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [70]:
import os
import os.path as osp
import random

import mlflow
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data
from torch.autograd import Variable
from torch.nn.utils import clip_grad_norm_
from tqdm import tqdm_notebook

from nlpclass.config import model_config
from nlpclass.data.data_utils import TranslationDataset, text_collate_func
from nlpclass.models.evaluation_utils import bleu_eval, output_to_translations
from nlpclass.models.models import DecoderRNN, EncoderCNN, EncoderRNN, TranslationModel
from nlpclass.models.training_utils import load_data

In [71]:
CURRENT_PATH = os.getcwd()
DATA_DIR = osp.join(CURRENT_PATH, '..', 'data')
MODEL_DIR = osp.join(CURRENT_PATH, '..','models')

# Data

In [72]:
data, data_loaders, max_length = load_data('vi', batch_size=24, subsample=0.025)

Counting words...
Counted words:
eng 6801
vi 7936
Counting words...
Counted words:
eng 3562
vi 3678
Counting words...
Counted words:
eng 3361
vi 3518


In [54]:
encoder = EncoderCNN(
    data['train'].input_lang.n_words,
    embedding_size=100,
    hidden_size=128,
    num_layers=2).to(model_config.device)
decoder = DecoderRNN(
    embedding_size=100,
    hidden_size=128,
    output_size=data['train'].output_lang.n_words,
    attention=False).to(model_config.device)
translation_model = TranslationModel(encoder, decoder, teacher_forcing_ratio=0.5).to(model_config.device)

In [73]:
encoder = EncoderRNN(
    data['train'].input_lang.n_words,
    embedding_size=100,
    hidden_size=128,
    num_layers=2,
    dropout=0.0,
    bidirectional=True).to(model_config.device)
if encoder.bidirectional:
    multiplier = 2
else:
    multiplier = 1
decoder = DecoderRNN(
    embedding_size=100,
    hidden_size=multiplier * 128,
    output_size=data['train'].output_lang.n_words,
    attention=True).to(model_config.device)
translation_model = TranslationModel(encoder, decoder, teacher_forcing_ratio=0.5).to(model_config.device)

In [74]:
optimizer = torch.optim.Adam(
            filter(lambda p: p.requires_grad, translation_model.parameters()), 1e-3)

In [75]:
weight = torch.ones(translation_model.decoder.output_size).to(model_config.device)
weight[model_config.PAD_token] = 0
criterion = nn.CrossEntropyLoss(weight)

In [76]:
def calc_loss(logits, target, criterion):
    logits_flat = logits.view(-1, logits.size(-1))
    target_flat = target.view(-1, 1).squeeze()
    return criterion(logits_flat, target_flat)

In [ ]:
for i in tqdm_notebook(range(25)):
    for batch in data_loaders['train']:
        optimizer.zero_grad()
        logits = translation_model(batch)
        loss = calc_loss(logits, batch['target'], criterion)
        loss.backward()
        
        encoder_norm = 0
        for p in translation_model.encoder.parameters():
            param_norm = p.grad.data.norm(2)
            encoder_norm += param_norm.item() ** 2
        decoder_norm = 0
        for p in translation_model.decoder.parameters():
            param_norm = p.grad.data.norm(2)
            decoder_norm += param_norm.item() ** 2
            
        print(encoder_norm, decoder_norm, loss.item())
            
        clip_grad_norm_(filter(lambda p: p.requires_grad,
                                   translation_model.parameters()), 5.0)
        
        optimizer.step() 
    original, translation = evaluate(translation_model, data, data_loaders, dataset_type='train')

0.08011834071292881 5.5834483141761995 9.032713890075684
0.02544476404865168 0.4136992865864504 8.94882583618164
0.013755178353915425 0.6389814253424574 8.893001556396484
0.012264135622871195 0.38159117100452694 8.844010353088379
0.0355917577045408 3.315481114308074 8.659758567810059
1.819223725631281 55.23717380686112 8.701007843017578
0.08138388873543415 2.5120004225334136 8.248764038085938
0.04675978511826759 2.6265302628131506 8.035236358642578
0.302169420343491 11.596012183430568 7.979457855224609
0.44847847658205786 12.388985829140159 7.775784492492676
0.6994426450752721 10.431422834903374 7.521955490112305
0.0885074192528298 5.6122538573460865 7.609282970428467
0.7281639397803501 20.270746908090096 7.336188793182373
0.04545427580976309 6.940355226910115 7.455748081207275
0.07757803705106614 2.1741109584888445 7.5292792320251465
0.1337557438770403 2.5942435657699514 7.093649387359619
0.10540119331257035 8.435210164002033 7.2331671714782715
1973.2125335509093 26603.93325657647 7.1

0.02147169077129879 0.4731992857289302 6.226560592651367
0.01664877080117559 0.30057885845571697 5.9254150390625
0.016480073150681975 0.3267396434573063 6.07928991317749
0.01076119070659073 0.3748816976676965 5.833324432373047
0.03133910604031902 0.316507770808708 6.001854419708252
0.028658827856451647 0.41632504982002994 6.216949939727783
0.048051538744987535 0.3616948959262559 6.19667387008667
0.03187615357334121 0.5612514946307391 5.976779460906982
0.03256820017379192 0.6144903821747661 6.162935256958008
0.06958742228878077 0.8586802927057413 6.119426250457764
0.04662971974753141 0.4230499679727256 5.985744953155518
0.016853093512267638 0.44266947161997233 5.676562309265137
0.022206470254063054 0.4005808395164131 6.018113613128662
0.02117184086149815 0.5060421883533152 6.163135528564453
0.015411106232180978 0.26909491829390425 6.167085647583008
0.01705997171190452 0.35278852270973354 5.991724967956543
0.04579636314349188 0.4317419084178983 6.093748092651367
0.01688464328762125 0.348

0.04548152926168607 0.3126989473668786 5.991243362426758
0.03570376183874302 0.42705160410750603 5.920356750488281
0.04847338217480229 0.4087248581300735 5.819079399108887
0.07642904041552816 0.37058944824271123 5.553945064544678
0.03465255421391583 0.3926142183665622 5.534770965576172
0.03338297975037584 0.3487265607257928 5.920706748962402
0.03191233769345314 0.398566494578827 5.468169689178467
0.04760762937920317 0.3843940569664467 5.898802757263184
0.03373056986334713 0.3109796350856586 6.06578254699707
0.038645977088557354 0.32994572118747034 5.962935924530029
0.040463774840590744 0.44611672303128913 5.9615325927734375
0.04989377477051894 0.385907642126788 5.31013298034668
0.05945526261058916 0.6345014817263822 5.733057498931885
0.037514178830692683 0.382140429826982 5.2933549880981445
0.03812957084194925 0.4816397810206702 5.477727890014648
0.045440485616722384 0.5625029929931246 5.308313369750977
0.03566501668672377 0.3215226498254041 5.675972938537598
0.040180337469557126 0.374

0.08495242431391432 0.5521070057637381 5.465666770935059
0.07561559683251545 0.49719496941043356 5.685389041900635
0.0842472944802816 0.47254616154561463 5.145084381103516
0.07868085390332981 0.505710007219584 5.440498352050781
0.10396116631796064 0.48567511944980335 5.542561054229736
0.09609371892337398 0.4410830946001608 5.5208258628845215
0.09296790800494403 0.5880535837446079 5.395535469055176
0.09957682394769979 0.463936074537356 5.744939804077148
0.0836014261910848 0.4024411932574771 5.215656280517578
0.0640003195173893 0.5565622333519825 5.355592727661133
0.12065058454997116 0.5231128526161475 5.369301795959473
0.1110855437137811 0.5950430289910649 4.77659273147583
0.08122945536254395 0.7135637740706255 5.087138652801514
0.07810184285895334 0.41872920204760317 5.322890281677246
0.07911360635614809 0.5302699304082974 5.563244819641113
0.0839680185174429 0.6003313695206985 5.405147552490234
0.07665583452572361 0.4473188401706804 5.181759357452393
0.061444690750905556 0.42411458827

0.10474970100190609 0.5835376572950778 4.9134650230407715
0.15232066450503523 0.5964342424812415 4.340785026550293
0.14197042669881976 0.6007161562573624 5.142252445220947
0.26350021769480086 0.886880813553926 4.345211982727051
0.08577979787996565 0.508528216337304 4.26238489151001
0.10438086826189476 0.6697469225928229 5.1545023918151855
0.13420002384533025 0.4873877844961632 4.38618803024292
0.10169006405274854 0.5532707049506492 4.362267971038818
0.10466012833769622 0.6246017486770208 4.389727592468262
0.12097786232631212 0.7061305977397704 4.848175048828125
0.3846577497958255 1.7457351380880428 4.883600234985352
0.14267521340553754 0.5316274851338813 4.500580310821533
0.12646882121320085 0.6869890727671958 5.115326881408691
0.14817532439912923 0.6881443005936525 5.096531391143799
0.15802162134670442 0.8616131018438093 4.112245082855225
0.11822567846399977 0.609685843255262 5.032459259033203
0.1369271444501695 0.711132965914597 4.610099792480469
0.21826746019011128 0.904808251125534

...

0.3216685021429487 0.7187984784621623 2.9115898609161377
0.2892214414419361 0.7231720310689881 2.809767961502075
0.24191179335626184 0.7306557647511097 3.095297336578369
0.23351582526088382 0.5292008197399164 1.7078263759613037
0.3137504050695739 0.6150913106535375 1.6131868362426758
0.209809521303838 0.5015526848326086 1.7004632949829102
0.26864737670519995 0.678548132019723 2.936218738555908
0.21595178268706217 0.46355976509120267 1.7161221504211426
0.26321891679914977 0.6802488045510231 2.752729654312134
0.23035398970933754 0.47882057139886447 1.550074815750122
0.3353782132660499 0.7897859305968169 3.0662598609924316
0.37617947525729806 0.5879690832191486 1.5876792669296265
0.34658156141866003 0.5937094705704634 1.3530218601226807
0.32834915971560297 0.7023958068212551 2.834784507751465
0.26265911245616935 0.700313398746061 3.2693593502044678
0.3399118884963428 0.742931328938016 2.208432197570801
0.363826541661237 0.6644410684269627 1.6668298244476318
1.0050503812252027 0.8326411881

0.29279831998613765 0.494710963116534 1.6339725255966187
0.2413338007001148 0.5661423978085787 1.5167454481124878
0.25278919067096106 0.490961288833588 1.482650876045227
0.2732401808005534 0.5431281646950621 1.5312168598175049
0.2078715485923156 0.5433341696093681 1.4675394296646118
0.26391308797532276 0.4693631809204617 1.6189523935317993
0.26623915930035735 0.4989735136857257 1.591867208480835
0.35506936890989044 0.584048564657718 1.5184521675109863
0.24431565628530175 0.45428004573160374 1.6287438869476318
0.1816950784469905 0.4360968861664772 1.496753454208374
0.20756312988587233 0.47037274033875653 1.3966015577316284
0.31403652113175795 0.640671155778991 1.470545768737793
0.23266737940509724 0.45129030727075087 1.4588913917541504
0.26380055356157206 0.7853636729834241 2.9612414836883545
0.2634449056919188 0.5433132517810437 1.5032391548156738
0.3518454938050858 0.867971836569283 3.0956718921661377
0.33212483021184347 0.7667833946635353 3.0299291610717773
0.21892110919955288 0.4764

0.24469777437957455 0.7013402043088269 3.027203321456909
0.2665001555682883 0.5858301109500684 3.327253818511963
0.2636756632588709 0.6572390392821191 2.8210043907165527
0.2609653436504069 0.4518038557638092 1.650213599205017
0.29319705646838556 0.48473524417132213 1.3561921119689941
0.4291298780008747 0.8995248169911578 3.2664852142333984
0.3023646820026259 0.7222022017927714 3.333486318588257
0.2876120695245243 0.5695018070852711 1.5797066688537598
0.34506920006088 0.6023142071040455 1.4333826303482056
0.7166052555593583 0.7905198378315614 2.861208200454712
0.2873995887574681 0.5195955043972108 1.96000075340271
0.29600501508699784 0.5705889934089684 1.3977296352386475
0.28385981879882005 0.4168792909247612 1.3868950605392456
0.3497109321940747 0.6952309825450681 3.014159679412842
0.34874133028288723 0.7328609048036794 2.1567790508270264
0.21844660825606685 0.451111741229506 1.3359718322753906
0.2533562396645888 0.6509687353618347 1.7550112009048462
0.44038310662408 0.9067577568334724

0.2734363327013778 0.6016477921722245 3.0450680255889893
0.34943360840172555 0.7576241820424902 2.9321746826171875
0.41265171555324676 0.6060487856760277 1.4699819087982178
0.30944609413581453 0.5598793349669399 1.5265865325927734
0.2919152894143039 0.5369143148619244 1.4242607355117798
0.20952789115384243 0.4580690303593021 1.263291835784912
0.3291570648699491 0.48047926013023645 1.3721157312393188
0.24304151461097268 0.508382175216161 1.5309851169586182
0.26856125195954883 0.4779736954265893 1.257122278213501
0.3256158368716045 0.7665495018410312 2.3490853309631348
0.5911296957756293 1.1589625316905379 2.398256540298462
0.2438861601397508 0.4504399461921729 1.5008612871170044
0.4102564701089061 0.88766601045373 3.3135323524475098
0.31771953329184316 0.6088829870182635 3.002979278564453
0.31415122067081214 0.5370119930945743 1.2450100183486938
0.4740365991856284 1.2184649469825863 3.0490851402282715
0.29006402143849813 0.5069386849328305 1.4879058599472046
0.32540336613100956 0.776774

0.24536809349306044 0.4937162613975592 1.2276644706726074
0.22863619765216073 0.4138799478295522 1.3134894371032715
0.2984546384020607 0.6728288971602636 2.4950475692749023
0.2533584243051314 0.45472666017503494 1.2561646699905396
0.3285697866344345 0.48235790097618847 1.4055047035217285
0.27376127381396875 0.6499204179180219 2.719494104385376
0.2334441897388192 0.4747664286723065 1.6006345748901367
0.29848568625863403 0.4900593725959232 1.1374458074569702
0.2861771589553138 0.6796232451994787 2.7469546794891357
0.3052736780575204 0.5502072288771301 1.2826985120773315
0.3724209011039312 0.6244090470383871 1.140214204788208
0.24801944564169853 0.44027333510966604 1.072554588317871
0.29152977369019306 0.6578481480749574 2.826955556869507
0.3096892733167446 0.534224086874115 1.525675892829895
0.4301980690219019 0.7549637315466139 2.5368075370788574
0.2661978962339267 0.4267925580599203 1.237884283065796
0.24044320742637262 0.46749607910221236 1.0369566679000854
0.20232758044437246 0.41907

In [66]:
x = next(iter(data_loaders['train']))
original = output_to_translations(x['target'], data['train'])
translations = output_to_translations(translation_model.greedy(x), data['train'])
print(bleu_eval(original, translations))
print(list(zip(translations[0:2], original[0:2])))

0.2907983665913475
[('and i think that you can do you can do it .', 'and a lot of that learning i think came from being on that farm because when i was working on the farm wed have to use what was around us wed have to use the environment and there was no such thing as something cant be done because youre in an environment where if you cant do what you need to do you can die and you know i had seen that sort of thing happen .'), ('throughout all a new york .', 'throughout all of this what i would ultimately realize was that each voice was closely related to aspects of myself and that each of them carried overwhelming emotions that id never had an opportunity to process or resolve memories of sexual trauma and abuse of anger shame guilt low self worth .')]


In [67]:
original = output_to_translations(x['target'], data['train'])
translations = output_to_translations(translation_model.beam_search(x), data['train'])
print(bleu_eval(original, translations))
print(translations)

TypeError: 'NoneType' object is not subscriptable

In [24]:
x = next(iter(data_loaders['train']))
input_seq = x['input']
target_seq = x['target']
input_length = x['input_length']
target_length = x['target_length']

encoded_input, hidden = translation_model.encoder.forward(input_seq, input_length)

In [25]:
encoded_input[0]

tensor([[-0.6125, -0.5094,  0.1136,  ...,  0.2936, -0.0332, -0.5566],
        [-0.2966, -0.5157,  0.0630,  ...,  0.6734,  0.0263, -0.9648],
        [ 0.3257, -1.1140,  0.3586,  ...,  0.7310, -0.7097, -0.5748],
        ...,
        [-0.3053, -0.1229,  0.2143,  ..., -0.5936, -0.5418, -0.0655],
        [-0.2868, -0.2641,  0.6324,  ..., -0.5235, -0.4746, -0.1487],
        [-0.0230, -0.0655,  0.1169,  ..., -0.4210,  0.3063,  0.4462]],
       device='cuda:0', grad_fn=<SelectBackward>)

In [26]:
x = next(iter(data_loaders['train']))

optimizer = torch.optim.Adam(
            filter(lambda p: p.requires_grad, translation_model.parameters()), 1e-2)

for i in range(100):
    optimizer.zero_grad()
    logits = translation_model(x)
    loss = calc_loss(logits, x['target'], criterion)
    print(loss)
    loss.backward()
    translation_model.encoder.embedding.weight.grad[data['train'].input_lang.pretrained_inds] = 0
    translation_model.decoder.embedding.weight.grad[data['train'].output_lang.pretrained_inds] = 0
    clip_grad_norm_(filter(lambda p: p.requires_grad,
                               translation_model.parameters()), model_config.grad_norm)
    optimizer.step()

tensor(10.8560, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(9.9659, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(8.5004, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(7.8864, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(6.6769, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(6.1789, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(6.1030, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(6.1107, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(5.3734, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(5.3272, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(4.8524, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(4.8219, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(4.5111, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(4.5380, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(4.2955, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(4.0173, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(4.1326, device='cuda:0', grad_fn=<NllLossBackwar

In [59]:
def evaluate(model, data, data_loaders, dataset_type='dev', max_batches=100):
    model.eval()
    with torch.no_grad():
        original_strings = []
        translated_strings = []
        for i, batch in enumerate(data_loaders[dataset_type]):
            if i > max_batches:
                break
            logits = translation_model(batch)
            epoch_loss = calc_loss(logits, batch['target'], criterion)
            original = output_to_translations(batch['target'], data['train'])
            translations = output_to_translations(model.greedy(batch), data['train'])
            original_strings.extend(original)
            translated_strings.extend(translations)
        bleu = bleu_eval(original_strings, translated_strings)
        model.train()
        print(epoch_loss)
        print(bleu)
        
        return original_strings, translated_strings

In [29]:
optimizer = torch.optim.Adam(
            filter(lambda p: p.requires_grad, translation_model.parameters()), 1e-3)
weight = torch.ones(translation_model.decoder.output_size).to(model_config.device)
weight[model_config.PAD_token] = 0
criterion = nn.CrossEntropyLoss(weight)

In [30]:
for i, batch in enumerate(tqdm_notebook(data_loaders['train'])):
    if i % 500 == 0:
        evaluate(translation_model, data, data_loaders)
        #evaluate(translation_model, data, data_loaders, dataset_type='train')
    optimizer.zero_grad()
    logits = translation_model(batch)
    loss = calc_loss(logits, batch['target'], criterion)
    loss.backward()
    translation_model.encoder.embedding.weight.grad[data['train'].input_lang.pretrained_inds] = 0
    translation_model.decoder.embedding.weight.grad[data['train'].output_lang.pretrained_inds] = 0
    clip_grad_norm_(filter(lambda p: p.requires_grad,
                               translation_model.parameters()), model_config.grad_norm)
    optimizer.step()

tensor(19.1450, device='cuda:0')
0.0008142795832761162
tensor(9.0464, device='cuda:0')
0.0033784249080402885


KeyboardInterrupt: 

In [ ]:
original_strings, translated_strings = evaluate(translation_model, data, data_loaders)

In [13]:
original_strings[0:10]

['baseball be later but of volumes all. hope. answer. only were',
 'be this. 65. know what must truth. i most',
 'games be practical everybody pins your needles all forward careful were',
 'i moment. it attack. be sorrows. ten.',
 'canadian be appointment careful laugh. no truth. something time were',
 'beautiful cupboard. hot. wine these',
 'much. but how bitter really dishonesty fed recently',
 'applied all else fed to climate husbands. matter. key',
 'but they answer. perfect. today. tape',
 'i is invented i situation getting powers your few']

In [14]:
translated_strings[0:10]

['i you you to the',
 'i you you to the',
 'i you you to the',
 'i you you to the',
 'i you you to the',
 'i you you to the',
 'i you you to the',
 'i you you to the',
 'i you you to the',
 'i you you to the']

In [48]:
decoder_input.size()

torch.Size([16])

In [23]:
total_loss.backward()

In [30]:
predictions = translation_model.greedy(x)

In [215]:
for row in predictions.cpu().numpy():
    decoded_words = []
    for elem in row[1:]:
        decoded_words.append(data['train']['output_lang'].index2word[elem])
        if elem == model_config.EOS_token:
            break

In [167]:
yo = Variable(torch.LongTensor([model_config.SOS_token] * 8)).to(model_config.device)
yo = torch.stack((yo, topi.squeeze(), topi.squeeze()), dim=1)

In [99]:
total_loss.backward()

In [57]:
seq_range = torch.autograd.Variable(torch.LongTensor(np.repeat([2], len(x['input_length'])))).to(model_config.device)
mask = seq_range < x['input_length']
loss = -torch.gather(decoder_output, dim=1, index=input_var.unsqueeze(1)).squeeze() * mask.float()

In [66]:
loss.sum() / torch.sum(loss > 0).float()

tensor(11.4193, device='cuda:0', grad_fn=<DivBackward1>)

In [63]:
torch.sum(loss > 0).cpu().numpy()

array(8)

In [ ]:
encoder_output, encoder_hidden = encoder(x['input'], x['input_length'])

In [290]:
context = None
if decoder.attention:
    context = Variable(torch.zeros(encoder_output.size(0), encoder_output.size(2))).unsqueeze(1).to(model_config.device)

In [291]:
decoder_output, decoder_hidden, context, weights = decoder(input_var, encoder_hidden, encoder_output, context)

In [16]:
def train_model(model, optimizer, train_loader, criterion):
    model.train()
    loss_train = 0
    for batch in train_loader:
        optimizer.zero_grad()
        outputs = model(batch)
        loss = criterion(outputs, batch['label'])
        loss.backward()
        optimizer.step()
        loss_train += loss.item() * \
            len(batch['label']) / len(train_loader.dataset)
    return loss_train

In [17]:
optimizer = torch.optim.Adam(translation_model.parameters(), lr=1e-3)
criterion = torch.nn.CrossEntropyLoss()
train_model(translation_model, optimizer, train_loader, criterion)

KeyError: 'label'